## pandas

### Getting Started

In [1]:
import numpy as np
import pandas as pd


df = pd.DataFrame({
    "quarter": [1, 1, 1, 2, 2, 2, 3, 3, 3, 4, 4, 4],
    "x": np.random.randn(12),
    "date": pd.date_range("2024-01-01", periods=12, freq="MS")
})

### Estonian Accidents Data

In [2]:
accidents = pd.read_csv("data/estonia-traffic-accidents-clean.csv")

#### Inspecting

In [3]:
accidents.shape

(14259, 8)

In [4]:
accidents.head()

,date,persons_involved,killed,injured,county,pedestrian_involved,accident_type,light_conditions
0,2014-10-24 08:45:00,2,0,1,Harju maakond,1,Jalakäijaõnnetus,Valge aeg
1,2014-10-24 13:45:00,2,0,1,Harju maakond,0,Kokkupõrge,Valge aeg
2,2014-08-11 00:00:00,2,0,1,Harju maakond,0,Kokkupõrge,Valge aeg
3,2014-11-17 17:32:00,2,0,2,Harju maakond,0,Kokkupõrge,Pimeda aeg
4,2015-04-28 07:55:00,2,0,1,Harju maakond,0,Kokkupõrge,Valge aeg


In [5]:
accidents.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14259 entries, 0 to 14258
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   date                 14259 non-null  object
 1   persons_involved     14259 non-null  int64 
 2   killed               14259 non-null  int64 
 3   injured              14259 non-null  int64 
 4   county               14259 non-null  object
 5   pedestrian_involved  14259 non-null  int64 
 6   accident_type        14259 non-null  object
 7   light_conditions     14259 non-null  object
dtypes: int64(4), object(4)
memory usage: 891.3+ KB


#### Selecting columns

In [6]:
accidents["date"]
accidents["date"].dtype

dtype('O')

We need to fix the data type (mutate)

In [7]:
accidents["date"] = pd.to_datetime(accidents["date"])

#### Pandas Series

In [8]:
# select and series
# What date range do the data cover?
dates = accidents["date"]
type(dates)
accidents["date"].min()
accidents["date"].max()

Timestamp('2021-12-31 23:45:00')

In [9]:
accidents["accident_type"]
accidents["accident_type"].value_counts()

accident_type
Kokkupõrge            5605
Ühesõidukiõnnetus     3946
Jalakäijaõnnetus      3386
Muu liiklusõnnetus    1262
Teadmata                60
Name: count, dtype: int64

#### Filter columns

How many of the accidents were in Harju county?

In [10]:
# filter
# How many of the accidents were in Harju county?
accidents["county"] == "Harju maakond"
accidents[accidents["county"] == "Harju maakond"].shape

(7000, 8)

How many people were killed or injured, overall?

Mutate and summarize.

In [11]:
# mutate
# create a new column: killed + injured
accidents["killed_or_injured"] = accidents["killed"] + accidents["injured"]

sum(accidents["killed_or_injured"])
accidents["killed_or_injured"].sum()

np.int64(18021)

Grouping: how many people were harmed, by accident type?


In [12]:
# summarize total accidents by something
by_type = accidents.groupby("accident_type").agg({"killed_or_injured": "sum"})
by_type


,killed_or_injured
accident_type,
Jalakäijaõnnetus,3548
Kokkupõrge,7951
Muu liiklusõnnetus,1436
Teadmata,70
Ühesõidukiõnnetus,5016


What happened to the "accident_type" column?

pandas Index

In [13]:
# index
# what happened to accident type?
by_type.columns
by_type.index

Index(['Jalakäijaõnnetus', 'Kokkupõrge', 'Muu liiklusõnnetus', 'Teadmata',
       'Ühesõidukiõnnetus'],
      dtype='object', name='accident_type')

To further explore this, let's look at the number of people harmed, by year

In [14]:
# how many people killed over time?
foo = accidents.resample(rule="YE", on="date").sum()
by_year = accidents.loc[:, ["date", "killed_or_injured", "persons_involved"]].resample(rule="YE", on="date").sum()

# index
# what happened to date?
by_year.columns
by_year.index

by_year = by_year.reset_index()
type(by_year["date"])
by_year["date"].dt.year
by_year["year"] = by_year["date"].dt.year
by_year = by_year.drop("date", axis=1)
by_year = by_year[["year", "persons_involved", "killed_or_injured"]]


For joins, merge in county population and look at rate
TODO

In [15]:
# TODO
# merge in country population, summarize by county rate



Reshaping / pivoting

for this we will look at another dataset

In [16]:

# for pivoting, use one of the health datasets
reflectors = pd.read_csv("data/reflectors.csv", skiprows=2, header=1, sep=";")


## Clean the data

In [17]:
accidents = pd.read_csv("data/estonia-traffic-accidents.csv")

accidents.head()

accidents.info()

accidents["Toimumisaeg"]

keep = ["Toimumisaeg", "Isikuid", "Hukkunuid", "Vigastatuid", "Maakond (PPA)",
        "Jalakäija osalusel", "Liiklusõnnetuse liik [1]", "Valgustus [1]"]

accidents = accidents[keep]

translate_columns = {"Toimumisaeg": "date", "Isikuid": "persons_involved", 
                     "Hukkunuid": "killed", "Vigastatuid": "injured", 
                     "Maakond (PPA)": "county", 
                     "Jalakäija osalusel": "pedestrian_involved", 
                     "Liiklusõnnetuse liik [1]": "accident_type", 
                     "Valgustus [1]": "light_conditions"}

accidents = accidents.rename(columns=translate_columns)

# fix date
import dateparser
accidents["date"] = accidents["date"].apply(lambda x: dateparser.parse(x, languages=["en"]))

# talk about missing values and filter those out
accidents.isna().sum()

accidents = accidents.dropna()

# talk about data types and fix those
accidents = accidents.astype({"persons_involved": "int64",
                              "pedestrian_involved": "int64"})

# accidents.to_csv("data/estonia-traffic-accidents-clean.csv", index=False)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15708 entries, 0 to 15707
Data columns (total 53 columns):
 #   Column                                         Non-Null Count  Dtype  
---  ------                                         --------------  -----  
 0   Juhtumi nr                                     15708 non-null  object 
 1   Toimumisaeg                                    15708 non-null  object 
 2   Isikuid                                        15633 non-null  float64
 3   Hukkunuid                                      15708 non-null  int64  
 4   Vigastatuid                                    15708 non-null  int64  
 5   Sõidukeid                                      15633 non-null  float64
 6   Aadress (PPA)                                  14601 non-null  object 
 7   Maja nr (PPA)                                  6686 non-null   object 
 8   Tänav (PPA)                                    14012 non-null  object 
 9   Ristuv tänav (PPA)                             275

## polars


In [18]:
import polars as pl

Pro-tip: always plot your data

In [19]:
# For this we also need to import the altair plotting library.
import altair as alt

# redo the basic examples from above

# datasuraus stuff from above
df1 = pl.read_csv("data/dataset1.csv")
df2 = pl.read_csv("data/dataset2.csv")



The data appear to be similar

In [20]:
df1.shape
df2.shape

(142, 2)

In [21]:
df1.describe()
df2.describe()

statistic,x,y
str,f64,f64
"""count""",142.0,142.0
"""null_count""",0.0,0.0
"""mean""",54.263273,47.832253
"""std""",16.765142,26.935403
"""min""",22.3077,2.9487
"""25%""",44.1026,25.2564
"""50%""",53.5897,46.0256
"""75%""",64.8718,69.1026
"""max""",98.2051,99.4872


In [22]:

df1.plot.point("x", "y")
df2.plot.point("x", "y")

alt.Chart(...)

## Appendix

In [23]:
# split this file but just use it to make a point about also plotting data
datasaurus = pd.read_csv("data/datasaurus.csv")

df1 = datasaurus[datasaurus["dataset"] == "away"].drop("dataset", axis=1)
df2 = datasaurus[datasaurus["dataset"] == "dino"].drop("dataset", axis=1)

df1.to_csv("data/dataset1.csv", index=False)
df2.to_csv("data/dataset2.csv", index=False)